In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fm2p
import cv2
from tqdm import tqdm

Loading DLC 2.3.7...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


c:\Users\dmartins\anaconda3\envs\fm1\lib\site-packages\oasis\functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [2]:
basepath = r'K:\Mini2P\241227_DMM_DMM037_freemoving'
rec_name = '241227_DMM_DMM037_freemoving_01'
session_props = r'K:\Mini2P\241227_DMM_DMM037_freemoving\session_props.json'

In [3]:
# reye = fm2p.Eyecam(basepath, rec_name)
# reye.find_files()
# ellipse_fit_results = reye.track_pupil()
# reye.save_tracking(ellipse_fit_results)

In [4]:
top = fm2p.Topcam(basepath, rec_name, props=session_props, rnum=1)
# top.find_files()
body_xyl, body_tracking_results = top.track_body()
topvid = fm2p.pack_video_frames(top.top_avi)
top.save_tracking(body_tracking_results, body_xyl, topvid)

100%|██████████| 12804/12804 [08:15<00:00, 25.85it/s]


In [6]:
twop = fm2p.TwoP(basepath, rec_name, props=session_props, rnum=1)
# twop.find_files()
twop_dict = twop.calc_dFF()
twop.save_fluor(twop_dict)

In [16]:
cmap = plt.cm.rainbow(np.linspace(0,1,360))

In [ ]:
plt.figure(figsize=(8,8))
for f in range(len(body_tracking_results['head_yaw'])):

    if np.isnan(body_tracking_results['head_yaw'][f]):
        continue
    
    plt.plot(
        body_tracking_results['x'][f],
        body_tracking_results['y'][f],
        '.',
        color=cmap[(360-int(np.round(np.rad2deg(body_tracking_results['head_yaw'][f]))))%360]
    )

In [ ]:
int(np.round(np.rad2deg(body_tracking_results['head_yaw'][f])))

In [ ]:
print(body_tracking_results['x'][f], body_tracking_results['y'][f])

In [ ]:
f = 2051

plt.plot(
    body_tracking_results['x'][f],
    body_tracking_results['y'][f],
    'o',
    color=cmap[(360-110)%360]
)

In [ ]:
plt.figure(figsize=(7,3))
plt.plot(body_tracking_results['topT']-body_tracking_results['topT'][0], body_tracking_results['head_yaw_deg'], '.', ms=1, color='k', linewidth=1)
plt.xlabel('sec')
plt.ylabel('yaw (deg)')
# plt.ylim([0, 40])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(7,3))
plt.plot(body_tracking_results['topT'][:-1]-body_tracking_results['topT'][0], body_tracking_results['speed'], '.', ms=1, color='k', linewidth=1)
plt.xlabel('sec')
plt.ylabel('cm/sec')
plt.ylim([0, 40])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(body_tracking_results['speed'], bins=np.linspace(0, 40, 12), color='k')
plt.xlim([0,40])
plt.xlabel('cm/s')
plt.title('frac moving = {:.3}%'.format(np.sum(body_tracking_results['speed']>2.)/len(body_tracking_results['speed'])*100))
plt.tight_layout()

In [ ]:
vidarr = fm2p.pack_video_frames(r'K:\FreelyMovingEyecams\241204_DMM_DMM031_freelymoving\241204_DMM_DMM031_freelymoving_01_top.avi')

In [5]:
xyl, _ = fm2p.open_dlc_h5(top.top_dlc_h5)
x_vals, y_vals, likelihood = fm2p.split_xyl(xyl)

# Threshold by likelihoods
x_vals = fm2p.apply_liklihood_thresh(x_vals, likelihood)
y_vals = fm2p.apply_liklihood_thresh(y_vals, likelihood)

In [ ]:
vid_save_path = 'demo_topdown_video.avi'

start = 1000
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_vid = cv2.VideoWriter(vid_save_path, fourcc, 60.0, (640, 480))
maxprev = 25

lear_x = x_vals['left_ear_x']
rear_x = x_vals['right_ear_x']
lear_y = y_vals['left_ear_y']
rear_y = y_vals['right_ear_y']
neck_x = x_vals['top_skull_x']
neck_y = y_vals['top_skull_y']
back_x = x_vals['base_tail_x']
back_y = y_vals['base_tail_y']
head_yaw = np.deg2rad(body_tracking_results['head_yaw_deg'])
body_yaw = np.deg2rad(body_tracking_results['body_yaw_deg'])
x_disp = body_tracking_results['x_displacement']
y_disp = body_tracking_results['y_displacement']

for f in tqdm(range(start,start+3600)):

    fig = plt.figure()

    plt.imshow(vidarr[f,:,:].astype(np.uint8), cmap='gray')
    # plt.ylim([135,0])
    # plt.xlim([0,180])
    plt.axis('off')

    plt.plot(lear_x[f], lear_y[f], 'b*')
    plt.plot(rear_x[f], rear_y[f], 'b*')

    plt.plot([neck_x[f], (neck_x[f])+15*np.cos(head_yaw[f])],
                [neck_y[f],(neck_y[f])+15*np.sin(head_yaw[f])],
                '-', linewidth=2, color='cyan') # head yaw
    
    plt.plot([back_x[f], (back_x[f])-15*np.cos(body_yaw[f])],
                [back_y[f], (back_y[f])-15*np.sin(body_yaw[f])],
                '-', linewidth=2, color='pink') # body yaw
    
    for p in range(maxprev):

        prevf = f - p

        plt.plot(neck_x[prevf],
                    neck_y[prevf], 'o', color='tab:purple',
                    alpha=(maxprev-p)/maxprev) # neck position history
        
    # arrow for vector of motion
    if body_tracking_results['forward_run'][f]:
        movvec_color = 'tab:green'
    elif body_tracking_results['backward_run'][f]:
        movvec_color = 'tab:orange'
    elif body_tracking_results['fine_motion'][f]:
        movvec_color = 'tab:olive'
    elif body_tracking_results['stationary'][f]:
        movvec_color = 'tab:red'
    
    plt.arrow(neck_x[f], neck_y[f],
                x_disp[f]*3, y_disp[f]*3,
                color=movvec_color, width=1)
    
    # Save the frame out
    fig.canvas.draw()
    frame_as_array = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    frame_as_array = frame_as_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close()

    img = cv2.cvtColor(frame_as_array, cv2.COLOR_RGB2BGR)
    out_vid.write(img.astype('uint8'))

out_vid.release()

In [ ]:
img.shape